In [1]:
%reload_ext autoreload
%autoreload 2

import musx
import rtmidi
from custom_utils import *

In [2]:
# Look for available ports
midiout = rtmidi.MidiOut()
outports = midiout.get_ports()
print("available ports:", outports)

available ports: ['Microsoft GS Wavetable Synth 0', 'musx2reaper 1']


In [3]:
# Connecting to the virtual LoopMIDI port that has Reaper listening in on it
midiout.open_port(outports.index('musx2reaper 1'))

In [4]:
def cc_init(queue):
    queue.out.addevent(musx.MidiEvent.control_change(0, 32, 0)) # Ukulele Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(0, 33, 64)) # Ukulele volume -> 0 dB
    queue.out.addevent(musx.MidiEvent.control_change(2, 32, 48)) # Blown bottle 1 volume -> -14 dB
    queue.out.addevent(musx.MidiEvent.control_change(3, 32, 0)) # Blown bottle 2 Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(3, 33, 31)) # Blown bottle 2 volume -> -30 dB
    queue.out.addevent(musx.MidiEvent.control_change(4, 32, 0)) # Swell bottles Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(4, 33, 64)) # Swell bottle 1 panning -> center
    queue.out.addevent(musx.MidiEvent.control_change(4, 34, 64)) # Swell bottle 2 panning -> center
    queue.out.addevent(musx.MidiEvent.control_change(4, 35, 31)) # Swell bottles volume -> -30 dB
    queue.out.addevent(musx.MidiEvent.control_change(6, 32, 16)) # Cup flicks volume -> -14 dB
    yield 0

In [5]:
def ukulele(queue):
    queue.compose([
        [0, distribution(queue, chan=0, length=40, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=1)],
        [8, distribution(queue, chan=0, length=32, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [16, distribution(queue, chan=0, length=24, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [24, distribution(queue, chan=0, length=16, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [32, distribution(queue, chan=0, length=8, notes=musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [40, distribution(queue, chan=0, length=28, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [40, cc_linear(queue, chan=0, ctrl=32, length=16, start=0, end=64, low=0, high=127)], # Ramp up Supermassive mix
        [56, distribution(queue, chan=0, length=16, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [60, distribution(queue, chan=0, length=28, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [64, distribution(queue, chan=0, length=24, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [68, distribution(queue, chan=0, length=20, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [88, drunk_walker(queue, chan=0, length=64, notes=musx.scale(36, 15, 2, 2, 1, 2, 2, 2, 1), dur=0.5, drunk_stride=2)],
        [96, cc_linear(queue, chan=0, ctrl=33, length=16, start=64, end=56, low=0, high=127)], # Lower volume slightly
        ])
    yield 0

In [6]:
def flicks(queue):
    queue.compose([
        [0, cc_linear(queue, chan=6, ctrl=32, length=32, start=16, end=64, low=0, high=127)], # Increase volume slowly
        [0, drunk_walker(queue, chan=6, length=64, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=2, drunk_stride=4)],
        [8, drunk_walker(queue, chan=7, length=64, notes=musx.scale(72, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=3)],
        [16, drunk_walker(queue, chan=6, length=64, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=2)],
        [24, drunk_walker(queue, chan=7, length=64, notes=musx.scale(72, 15, 2, 2, 1, 2, 2, 2, 1), dur=0.5, drunk_stride=1)],
        [64, cc_linear(queue, chan=6, ctrl=32, length=24, start=64, end=32, low=0, high=127)], # Decrease volume slowly
    ])
    yield 0

In [7]:
def blown_wander(queue):
    queue.compose([
        [0, cc_linear(queue, chan=2, ctrl=32, length=20, start=48, end=64, low=0, high=127)], # Increase volume slowly
        [0, drunk_walker(queue, chan=2, length=96, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=2)],
        [8, drunk_walker(queue, chan=2, length=96, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=2)],
        [16, drunk_walker(queue, chan=2, length=96, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=2)],
        [24, drunk_walker(queue, chan=2, length=96, notes=musx.scale(60, 15, 2, 2, 1, 2, 2, 2, 1), dur=1, drunk_stride=2)]
    ])
    yield 0

In [8]:
def blown_bottles(queue):
    queue.compose([
        [0, cc_linear(queue, chan=3, ctrl=33, length=24, start=32, end=64, low=0, high=127)], # Increase volume slowly
        [20, cc_linear(queue, chan=3, ctrl=32, length=60, start=0, end=1)], # Increase Supermassive mix gradually
    ])

    for iteration in range(35):
        jumble_gen = musx.jumble(musx.scale(48, 23, 2, 2, 1, 2, 2, 2, 1))
        notes = [next(jumble_gen) for _ in range(6)]
        positions = 16
        for note in notes:
            events = round(musx.rescale(musx.midran(), 0, 1, 1, positions))
            rotation = round(musx.rescale(musx.uniran(), 0, 1, 0, positions - 1))
            queue.compose([[iteration * 4, euclidean_rhythm(queue, chan=3, length=4, keynum=note, events=events, positions=positions, cycletime=4, rotation=rotation)]])
    yield 0

In [9]:
def swell(queue):
    queue.compose([
        [0, cc_linear(queue, chan=4, ctrl=35, length=12, start=31, end=56, low=0, high=127)], # Increase volume slowly
        [0, random_ascend(queue, chan=4, length=128, notes=musx.scale(84, 17, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [0, random_ascend(queue, chan=5, length=128, notes=musx.scale(72, 17, 2, 2, 1, 2, 2, 2, 1), dur=1)],
        [32, cc_distribution(queue, chan=4, ctrl=33, length=96, rate=0.25)], # Randomize panning
        [32, cc_distribution(queue, chan=4, ctrl=34, length=96, rate=0.5)], # Randomize panning
        [48, cc_linear(queue, chan=4, ctrl=32, length=32, start=0, end=0.35)], # Increase Supermassive mix gradually
        [106, cc_linear(queue, chan=4, ctrl=32, length=24, start=0.35, end=1)] # Increase Supermassive mix gradually to maximum
    ])
    yield 0

In [12]:
# Generate all of the MIDI and store in seq
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([
    [0, cc_init(queue)],
    [0, ukulele(queue)],
    [104, flicks(queue)],
    [154, blown_wander(queue)],
    [202, blown_bottles(queue)],
    [240, swell(queue)],
])

Done!


In [13]:
# Send the MIDI sequence through the port to Reaper
seq.play(midiout, False)

In [20]:
# Restart the port, clearing all sound
midiout = midi_restart(midiout, 'musx2reaper 1')